### csvをftrに変換する
- jsonもパースして、一般的なdataframe形式に落とし込む
https://www.kaggle.com/johnfarrell/gacrp-data-to-feather
https://www.kaggle.com/sudalairajkumar/simple-exploration-baseline-ga-customer-revenue


In [2]:
import os
import gc
import feather
import numpy as np
import pandas as pd
import json
from pandas.io.json import json_normalize

In [3]:
def load_df(csv_path='../input/train.csv', nrows=None):
    JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
    
    df = pd.read_csv(csv_path, 
                     converters={column: json.loads for column in JSON_COLUMNS}, 
                     dtype={'fullVisitorId': 'str'}, # Important!!
                     nrows=nrows)
    
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    print(f"Loaded {os.path.basename(csv_path)}. Shape: {df.shape}")
    return df

In [4]:
%%time
train = load_df(csv_path='../data/input/train.csv')
test = load_df(csv_path='../data/input/test.csv')

Loaded train.csv. Shape: (903653, 55)
Loaded test.csv. Shape: (804684, 53)
CPU times: user 4min 53s, sys: 12.9 s, total: 5min 6s
Wall time: 5min 13s


In [5]:
%%time
feather.write_dataframe(train, '../data/interim/train.ftr')
feather.write_dataframe(test, '../data/interim/test.ftr')

CPU times: user 8.7 s, sys: 3.33 s, total: 12 s
Wall time: 11.9 s


In [8]:
%%time
train = feather.read_dataframe('../data/interim/train.ftr')
test = feather.read_dataframe('../data/interim/test.ftr')

CPU times: user 8.54 s, sys: 2.2 s, total: 10.7 s
Wall time: 10.7 s


In [9]:
train.head()

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserSize,...,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.campaign,trafficSource.campaignCode,trafficSource.isTrueDirect,trafficSource.keyword,trafficSource.medium,trafficSource.referralPath,trafficSource.source
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,None,None,None,(not set),None,None,(not provided),organic,None,google
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,None,None,None,(not set),None,None,(not provided),organic,None,google
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,None,None,None,(not set),None,None,(not provided),organic,None,google
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,None,None,None,(not set),None,None,google + online,organic,None,google
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,None,None,None,(not set),None,True,(not provided),organic,None,google
